In [ ]:
# Data Extraction Rest API

import os
import subprocess

# Set up your Kaggle API key
os.environ["KAGGLE_USERNAME"] = "Your_Kaggle_username_here"
os.environ["KAGGLE_KEY"] = "Your_API_Key_here"

# Download the dataset using the Kaggle API
subprocess.run(["kaggle", "datasets", "download", "-d", "dubradave/formula-1-drivers-dataset"])

# Optionally, unzip the downloaded file
subprocess.run(["unzip", "formula-1-drivers-dataset.zip", "-d", "formula-1-drivers-dataset"], check=True)

print("Dataset downloaded and unzipped successfully")


# Data Migration to PostgreSQL Server

import psycopg2
import pandas as pd

# Replace these with your actual PostgreSQL credentials and database details
DB_HOST = 'host'
DB_NAME = 'database'
DB_USER = 'username'
DB_PASS = 'password'
DB_PORT = 'port_number'  # Default is usually 5432

# Define the path to your CSV file
csv_file_path = 'Directory/F1/F1DriversDataset.csv'

try:
    # Establish the database connection
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )

    # Create a cursor object
    cur = conn.cursor()

    # Create a table (if it doesn't already exist)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS formula1_drivers (
        id SERIAL PRIMARY KEY,
        driver VARCHAR(100),
        nationality VARCHAR(100),
        seasons VARCHAR(255),
        championships FLOAT,
        race_entries FLOAT,
        race_starts FLOAT,
        pole_positions FLOAT,
        race_wins FLOAT,
        podiums FLOAT,
        fastest_laps FLOAT,
        points FLOAT,
        active BOOLEAN,
        championship_years VARCHAR(255),
        decade INTEGER,
        pole_rate FLOAT,
        start_rate FLOAT,
        win_rate FLOAT,
        podium_rate FLOAT,
        fastlap_rate FLOAT,
        points_per_entry FLOAT,
        years_active INTEGER,
        champion BOOLEAN
    )
    """
    cur.execute(create_table_query)
    conn.commit()

    # Read data from the CSV file
    df = pd.read_csv(csv_file_path)

    # Insert data from the DataFrame into the PostgreSQL table
    for _, row in df.iterrows():
        cur.execute(
            """
            INSERT INTO formula1_drivers (
                driver, nationality, seasons, championships, race_entries, race_starts, pole_positions,
                race_wins, podiums, fastest_laps, points, active, championship_years, decade, pole_rate,
                start_rate, win_rate, podium_rate, fastlap_rate, points_per_entry, years_active, champion
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['Driver'], row['Nationality'], row['Seasons'], row['Championships'], row['Race_Entries'],
                row['Race_Starts'], row['Pole_Positions'], row['Race_Wins'], row['Podiums'], row['Fastest_Laps'],
                row['Points'], row['Active'], row['Championship Years'], row['Decade'], row['Pole_Rate'],
                row['Start_Rate'], row['Win_Rate'], row['Podium_Rate'], row['FastLap_Rate'], row['Points_Per_Entry'],
                row['Years_Active'], row['Champion']
            )
        )

    # Commit the transaction
    conn.commit()

except Exception as error:
    print(f"Error: {error}")
finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

print("CSV data inserted successfully.")